In [1]:
import torch
import torchvision
import torchvision.models as models
from PIL import Image, ImageFile
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import gc; gc.enable()
import pandas as pd
from torch.utils.data import Dataset
import tensorflow as tf
from tensorflow import keras
from torchvision import transforms

In [2]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
# keras imports
from keras.models import Model
from keras.models import Sequential
from keras.models import model_from_json
from keras.layers import Input
# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import h5py
import json
import datetime
import time

Using TensorFlow backend.


AttributeError: module 'keras_applications' has no attribute 'set_keras_submodules'

In [3]:
train_dir = '/Users/flatironschool/Documents/Kaggle/Kaggle-DR-detection/data/train/'

In [4]:

xception = keras.applications.Xception(include_top = False, weights = 'imagenet')

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
input_shape = xception.output_shape
out = xception.output
out = keras.layers.GlobalAveragePooling2D()(out)
out = keras.layers.Dense(512, activation='relu')(out)
total_classes = 5
predictions = keras.layers.Dense(total_classes, activation='softmax')(out)

model = keras.models.Model(inputs=xception.input, outputs=predictions)

In [6]:
set_trainable = False
cnt = 0
for layer in model.layers:
    if cnt>130:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
    cnt+=1

In [9]:
# training_generator = BalancedBatchGenerator(X_train, y_train,
#                                                 batch_size=1000,
#                                                 random_state=42)
#     model.fit_generator(generator=training_generator, epochs=5, verbose=1)
datagen = keras.preprocessing.image.ImageDataGenerator()
train_generator = datagen.flow_from_directory(train_dir, target_size = (224,224), classes = ['0','1','2','3','4'],class_mode ='categorical') 
batch_size = 32
train_steps_per_epoch = 30038//batch_size
adm = tf.keras.optimizers.Adam(lr = 0.08)
model.compile(optimizer = adm, loss = 'categorical_crossentropy')
history = model.fit_generator(train_generator,
                              steps_per_epoch=train_steps_per_epoch,
                              epochs=15, verbose=1)

Found 31067 images belonging to 5 classes.
Epoch 1/15
971/971 [==============================] - 6076s 6s/step - loss: 13.8922
Epoch 2/15
971/971 [==============================] - 6035s 6s/step - loss: 13.8922
Epoch 3/15
971/971 [==============================] - 5996s 6s/step - loss: 13.8926
Epoch 4/15
971/971 [==============================] - 5999s 6s/step - loss: 13.8922
Epoch 5/15
  8/971 [..............................] - ETA: 1:49:20 - loss: 14.2293

KeyboardInterrupt: 

In [10]:
plt.plot(history.history['loss'])
plt.savefig('loss_Xception_20190903')

NameError: name 'history' is not defined

In [11]:
model_json = model.to_json()
with open("Xception_rebalanced.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("Xception_rebalanced.h5")
print("Saved model to disk")

Saved model to disk


In [23]:
get_last_pretrained_layer = keras.backend.function([model.layers[0].input],
                                  [model.layers[131].output])

In [27]:
train_features = np.zeros(shape=(21972, 7, 7, 2048))
train_labels = np.zeros(shape=(21972,5))

In [30]:
i = 0
for inputs_batch, labels_batch in train_generator:
    features_batch = get_last_pretrained_layer(inputs_batch)[0]
    train_features[i : i + len(inputs_batch)] = features_batch
    train_labels[i : i +len(inputs_batch)] = labels_batch
    i += len(inputs_batch)
    print(i)
    if i+2 > len(train_features):
        break
         
train_features = np.reshape(train_features, (21972, 7 * 7 * 2048))

32
64
96
128
160
192
224
256
288
320
352
384
416
448
480
512
544
576
608
640
672
704
736
768
800
832
864
896
928
960
992
1024
1056
1088
1120
1152
1184
1216
1248
1280
1312
1344
1376
1408
1440
1472
1504
1536
1568
1600
1632
1664
1696
1728
1760
1792
1824
1856
1888
1920
1952
1984
2016
2048
2080
2112
2144
2176
2208
2240
2272
2304
2336
2368
2400
2432
2464
2496
2528
2560
2592
2624
2656
2688
2720
2752
2784
2816
2848
2880
2912
2944
2976
3008
3040
3072
3104
3136
3168
3200
3232
3264
3296
3328
3360
3392
3424
3456
3488
3520
3552
3584
3616
3648
3680
3712
3744
3776
3808
3840
3872
3904
3936
3968
4000
4032
4064
4096
4128
4160
4192
4224
4256
4288
4320
4352
4384
4416
4448
4480
4512
4544
4576
4608
4640
4672
4704
4736
4768
4800
4832
4864
4896
4928
4960
4992
5024
5056
5088
5120
5152
5184
5216
5248
5280
5312
5344
5376
5408
5440
5472
5504
5536
5568
5600
5632
5664
5696
5728
5760
5792
5824
5856
5888
5920
5952
5984
6016
6048
6080
6112
6144
6176
6208
6240
6272
6304
6336
6368
6400
6432
6464
6496
6528
6560
6592
6624

In [33]:
import gc; gc.enable()
del model, xception, train_generator; gc.collect()

3625

In [4]:
from sklearn.model_selection import train_test_split
feat_train, feat_test, labels_train, labels_test = train_test_split(train_features, train_labels, test_size = 0.2)

NameError: name 'train_features' is not defined

In [ ]:
oof_preds = np.zeros(train.shape[0])
sub_preds = np.zeros(test.shape[0])

feature_importance_df = pd.DataFrame()

In [32]:
from sklearn.model_selection import StratifiedKFold, KFold
import xgboost as xgb
params = {
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 5,
}

In [1]:
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train, y_train)):
    
    trn_x, trn_y = train[train_cols].iloc[trn_idx], y_train.iloc[trn_idx]
    val_x, val_y = train[train_cols].iloc[val_idx], y_train.iloc[val_idx]
    gc.collect()
    dtrain = xgb.DMatrix(trn_x, trn_y, feature_names=trn_x.columns)
    dval = xgb.DMatrix(val_x, val_y, feature_names=val_x.columns)
    gc.collect()
    
    clf = xgb.train(params=params, dtrain=dtrain, num_boost_round=1500, evals=[(dtrain, "Train"), (dval, "Val")],
        verbose_eval= 250, early_stopping_rounds=100) 
    gc.collect()
    
    oof_preds[val_idx] = clf.predict(xgb.DMatrix(val_x))
    sub_preds += clf.predict(xgb.DMatrix(test[train_cols])) / folds.n_splits
    gc.collect()
    
    xgbfir.saveXgbFI(clf, feature_names=trn_x.columns, OutputXlsxFile='ieee_xgbfir_%sFold.xlsx'%str(n_fold+1), MaxInteractionDepth=9, MaxHistograms=15)
    gc.collect()
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = pd.DataFrame.from_dict(data=clf.get_fscore(), orient="index", columns=["FScore"])["FScore"].index
    fold_importance_df["fscore"] = pd.DataFrame.from_dict(data=clf.get_fscore(), orient="index", columns=["FScore"])["FScore"].values
    fold_importance_df["fold"] = n_fold + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    gc.collect()
    
    print('\nFold %2d AUC %.6f & std %.6f' %(n_fold + 1, roc_auc_score(val_y, oof_preds[val_idx]), np.std([oof_preds[val_idx]])))
    gc.collect()

print('\nCV AUC score %.6f & std %.6f' % (roc_auc_score(y_train, oof_preds), np.std((oof_preds))))


NameError: name 'folds' is not defined

In [3]:
X = feat_train
bleh, y = np.where(labels_train)

xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
xgb_model.fit(X, y)

y_pred = xgb_model.predict(feat_test)

NameError: name 'feat_train' is not defined

In [ ]:
from sklearn.metrics import confusion_matrix, cohen_kappa_score
bleh, y_test = np.where(labels_test)
print(cohen_kappa_score(y_test, y_pred))